---
title: "2023_Kronensicherung_Bosau_Baumdaten"
author: "Kyell Jensen"
date: "2024-08-06"
format: pdf
editor: visual
---

%load_ext autoreload
%autoreload 2

%aimport treemotion

# 2023_Kronensicherung_Bosau_Baumdaten

## Analyse der TreeMotionSensor-Daten

Nutze eine geeignete Python 3.11 Umgebung (z. B. virtuelle Environment) und installiere das Paket TreeMotion (TMS) inklusive kj_core und kj_logger und weiteren requirements.

## Package Importe

Es werden zuerst benötigte Standard-Pakete importiert. Nachfolgend wird das extra erstellte TreeMotion-Package importiert. Fehler beim Import dieses Pakets sind ggf. Bugs. Das Paket nutzt eine gemeinsame CodeBasis in den Paketen kj_core (Core-Package) und kj_logger (individualisiertes Logging des Verarbeitungs-Prozesses). Diese sollte i. d. R. über die requirements mit installiert werden.

### Public imports


In [1]:
from pathlib import Path
from datetime import datetime, time

import numpy as np
import pandas as pd

## Import TreeMotion-Package

Das Paket TreeMotion (tms) wurde vom Autor (Kyell Jensen) zum einfachen Analysieren, Plotten und zur Interpretation der Messdaten der TreeMotionSensoren (Accelerometer/Inclinometer) der Firma "IML Instrumenta Mechanik Labor Electronic" (ehemals "Argus Electronic") geschrieben (https://www.iml-electronic.de/produkt/picus-treemotion/). Das Packet ist auf GitHub verfügbar (https://github.com/Kyellsen/treemotion). Die Messdaten der Geräte müssen für die Verarbeitung in Python erst in die Firmen eigene Software "TMS.Software" importiert werden. Von dort ist ein manueller Export in einem lesbaren CSV-Format möglich. Dieser Schritt lässt sich nicht automatisieren.
Nachfolgend wird das Packet und einige dort definierten Klassen importiert.

In [2]:
import treemotion as tms

# Test
# noinspection PyUnresolvedReferences
from treemotion import Project, Series, Measurement, MeasurementVersion, DataTMS, DataMerge, Tree, TreeCable
# noinspection PyUnresolvedReferences
#from treemotion import CrownMotionSimilarity

2025-05-07 15:44:18 [INFO] treemotion.classes.project.<module>: TEAST


## Daten Importe

Lege Pfade für Daten-Importe, Daten-Exporte etc. fest (ggf. anpassen an eigene Verzeichnisstruktur)

In [3]:
# Main
main_path = Path(r"C:\kyellsen\005_Projekte\2022_Bosau")
analyse_name = r"2022_Kronensicherung_Bosau_2024-02-06"
data_path = main_path / "021_Daten_Clean"  # Für alle Daten-Importe des Projektes gemeinsam
working_directory = main_path / "030_Analysen" / analyse_name / "working_directory"  # Für alle Daten-Exporte des Projektes gemeinsam
db_name = "TREEMOTION_Bosau_2023-05-12.db"
source_db = data_path / db_name
csv_path = r"C:\kyellsen\005_Projekte\2022_Bosau\020_Daten"

### Initialisieren des TreeMotionPackages

Für die Funktionalität des TMS-Packages werden div. Instanzen von Manager-Klassen erstellt. Hierfür wird die 'tms.setup' Funktion verwendet. Sie liest die Default-Config des Packages und erstellt entsprechend angepasster Manager.  

In [4]:
CONFIG, LOG_MANAGER, DATA_MANAGER, DATABASE_MANAGER, PLOT_MANAGER = tms.setup(
        working_directory=str(working_directory), log_level="DEBUG")

2025-05-07 15:44:18 [INFO] kj_logger.update_config: <kj_logger.LogManager object at 0x000001B9F8C97450> initialized - update_config! Code: 000
2025-05-07 15:44:18 [WARNING] kj_core.core_config.set_working_directory: The directory C:\kyellsen\005_Projekte\2022_Bosau\030_Analysen\2022_Kronensicherung_Bosau_2024-02-06\working_directory\tms already exists.
2025-05-07 15:44:18 [INFO] kj_core.core_config.set_working_directory: Working directory set to C:\kyellsen\005_Projekte\2022_Bosau\030_Analysen\2022_Kronensicherung_Bosau_2024-02-06\working_directory\tms!
2025-05-07 15:44:18 [INFO] kj_core.core_config.__init__: <CoreConfig>
  Package:            treemotion
  Working Directory:  C:\kyellsen\005_Projekte\2022_Bosau\030_Analysen\2022_Kronensicherung_Bosau_2024-02-06\working_directory\tms
  Plot Directory:     C:\kyellsen\005_Projekte\2022_Bosau\030_Analysen\2022_Kronensicherung_Bosau_2024-02-06\working_directory\tms\plots
  Data Directory:     C:\kyellsen\005_Projekte\2022_Bosau\030_Analyse

### Lade Metadaten zum Versuch (Sensorpositionierung, Versuchsprotokoll/Messablauf, Baumdaten)

Die Daten wurden während des Versuchs in einer SQLite-Datenbank gesammelt und strukturiert. Teilweise wurden Tabellen aus GoogleSheets manuell eingepflegt, da kletternd im Baum eine Erfassung in solchen deutlich anwenderfreundlicher ist.
Die Datenbank ist damit die zentrale stelle für alle Metadaten des Feldversuchs in Bosau.
Änderungen an der Ausgangs-Datenbank sollen vermieden werden - sie wird entsprechend einmalig ins 'working_directory' kopiert. Die weitere Analyse erfolgt dann auf der Kopie der Datenbank
Es folgt die Verbindung mit der Datenbank und die Erstellung einer 'Project'-Instance. Innerhalb des Projektes können dann bestimmte Messreihen ('Series') und Messungen ('Measurement').

In [5]:
DATABASE_MANAGER.duplicate(database_path=str(source_db)) # Only call ones

2025-05-07 15:44:18 [INFO] kj_core.utils.database_manager.duplicate: Database TREEMOTION_Bosau_2023-05-12.db duplicated to C:\kyellsen\005_Projekte\2022_Bosau\030_Analysen\2022_Kronensicherung_Bosau_2024-02-06\working_directory\tms\databases\TREEMOTION_Bosau_2023-05-12.db


In [6]:
DATABASE_MANAGER.connect(db_name=str(db_name))
project_1 = DATABASE_MANAGER.load(Project, 1)[0]
series_1: Series = DATABASE_MANAGER.load(Series, 1)[0]
m_1: Measurement = series_1.measurement[0]

2025-05-07 15:44:18 [INFO] kj_core.utils.database_manager.connect: Connecting to existing database at C:\kyellsen\005_Projekte\2022_Bosau\030_Analysen\2022_Kronensicherung_Bosau_2024-02-06\working_directory\tms\databases\TREEMOTION_Bosau_2023-05-12.db
2025-05-07 15:44:18 [INFO] kj_core.utils.database_manager.connect: Successfully connected!
2025-05-07 15:44:18 [DEBUG] kj_core.utils.database_manager.load: Prozess instance of 'Project' with primary key '1'
2025-05-07 15:44:19 [INFO] kj_core.utils.database_manager.load: Loading successful '1' instances of class Project.
2025-05-07 15:44:19 [DEBUG] kj_core.utils.database_manager.load: Prozess instance of 'Series' with primary key '1'
2025-05-07 15:44:19 [INFO] kj_core.utils.database_manager.load: Loading successful '1' instances of class Series.


Die Messdaten der TreeMotionSensoren wurden von den Geräten gesichert und aus der "TMS.Software" als CSV-Dateien exportiert. Die spätere Zuordnung erfolg entsprechend über die Messreihe (Series) und die SensorID. Hierzu steht im Package die Methode 'Series.add_filenames()' zur Verfügung. Sofern in der Datenbank für jeden 'Series' der richtige 'filepath_tms' zum Verzeichnis der zugehörigen TMS.csv-Dateien hinterlegt ist, ordnet Sie die Dateien richtig den Messungen zu.
Sie Methode wird nachfolgend für die Messreihe 1 aufgerufen. Die Methode 'Measurement.create_from_csv' liest dann die CSV-Daten ein und erstellt eine Version der Daten der Messung. Eine Messung kann entsprechend in verschiedenen Verarbeitungszuständen abgespeichert werden. Die Messdaten werden nachfolgend intern nicht in der Datenbank, sondern als .feather-Dateien mit Referenz in der Datenbank gespeichert (deutlich effizienter als CSV-Dateien).


In [7]:
series_1.add_filenames(csv_path=csv_path)
# series_1.method_for_all_of_class("Measurement", "load_from_csv", measurement_version_name="raw", update_existing=True)

2025-05-07 15:44:19 [INFO] kj_core.utils.path_utils.validate_and_get_file_list: Found files: ['2022-01-29 020000__DatasA000-0000-0007', '2022-01-29 020000__DatasA000-0000-0008', '2022-01-29 020000__DatasA000-0000-0010', '2022-01-29 020000__DatasA000-0000-0011', '2022-01-29 020000__DatasA000-0000-0012', '2022-01-29 020000__DatasA000-0000-0013', '2022-01-29 020000__DatasA000-0000-0014', '2022-01-29 020000__DatasA000-0000-0015', '2022-01-29 020000__DatasA000-0000-0016', '2022-01-29 020000__DatasTI0000000027', '2022-01-29 020000__DatasTI0000000075', '2022-01-29 020000__DatasTI0000000076', '2022-01-29 020000__DatasTI0000000077', '2022-01-29 020000__DatasTI0000000078', '2022-01-29 020000__DatasTI0000000080', '2022-01-29 020000__DatasTI0000000082', '2022-01-29 020000__DatasTI0000000083', '2022-01-29 020000__DatasTI0000000084', '2022-01-29 020000__DatasTI0000000085']
2025-05-07 15:44:19 [INFO] kj_core.utils.path_utils.extract_sensor_id: Extracted sensor IDs: [7, 8, 10, 11, 12, 13, 14, 15, 16, 

In [8]:
session = DATABASE_MANAGER.session

# Abfragen aller Daten aus der Tabelle 'Tree'
tree_data = session.query(Tree).all()

# Abfragen aller Daten aus der Tabelle 'TreeCable'
tree_cable_data = session.query(TreeCable).all()

# Erstellen eines DataFrames aus den abgefragten Daten für 'Tree'
tree_df = pd.DataFrame([{
    'Umfang': tree.circumference,
    'Höhe (ca.)': tree.height,
    'Vergabelungshöhe': tree.fork_height
} for tree in tree_data])

# Erstellen eines DataFrames aus den abgefragten Daten für 'TreeCable'
tree_cable_df = pd.DataFrame([{
    'Höhe KS': cable.height,
    'Länge KS': cable.length,
    'Umfang Stämmlinge A auf Höhe KS': cable.trunk_a_circumference,
    'Umfang Stämmlinge B auf Höhe KS': cable.trunk_b_circumference
} for cable in tree_cable_data])

# Kombinieren der Umfänge von Stämmen A und B in einer gemeinsamen Serie
combined_circumference = pd.concat([
    tree_cable_df['Umfang Stämmlinge A auf Höhe KS'], 
    tree_cable_df['Umfang Stämmlinge B auf Höhe KS']
])

metrics = ['min', 'mean', 'max', 'std']

# Statistische Auswertungen für die kombinierten Umfänge
circumference_statistics = combined_circumference.agg(metrics)
circumference_statistics = pd.DataFrame(circumference_statistics).transpose()
circumference_statistics.index = ['Umfang Stämmlinge auf Höhe KS']

# Statistische Auswertungen für die Spalten in 'Tree' DataFrame
tree_statistics = tree_df.agg(metrics).transpose()

# Statistische Auswertungen für die anderen Spalten in 'TreeCable' DataFrame
tree_cable_statistics = tree_cable_df[['Höhe KS', 'Länge KS']].agg(metrics).transpose()

# Zusammenführen der Statistiken in einem DataFrame
combined_statistics = pd.concat([tree_statistics, tree_cable_statistics, circumference_statistics])
combined_statistics["Einheit"] = "cm"

combined_statistics

,min,mean,max,std,Einheit
Umfang,217.0,234.900000,258.0,11.376877,cm
Höhe (ca.),2200.0,2200.000000,2200.0,0.000000,cm
Vergabelungshöhe,316.0,387.700000,510.0,55.643808,cm
Höhe KS,1481.0,1611.636364,1746.0,100.477134,cm
Länge KS,136.0,218.090909,290.0,41.810177,cm
Umfang Stämmlinge auf Höhe KS,47.0,56.181818,70.0,6.932576,cm


In [9]:
# Umwandlung der statistischen Auswertung in LaTeX
latex_string = combined_statistics.to_latex(index=True, escape=True, column_format="lrrrrr", header=["Min.", "Mean", "Max.", "Std.", "Einheit"], 
                                            float_format="{:0.2f}".format)

# LaTeX Tabellen-String formatieren
latex_table = f"""
\\begin{{table}}[h]
    \\centering
    {latex_string}
    \\caption{{Plesse - Daten der Versuchsbäume}}
    \\label{{tab:plesse_versuchsbaeume}}
\\end{{table}}
"""

print(latex_table)


\begin{table}[h]
    \centering
    \begin{tabular}{lrrrrr}
\toprule
 & Min. & Mean & Max. & Std. & Einheit \\
\midrule
Umfang & 217.00 & 234.90 & 258.00 & 11.38 & cm \\
Höhe (ca.) & 2200.00 & 2200.00 & 2200.00 & 0.00 & cm \\
Vergabelungshöhe & 316.00 & 387.70 & 510.00 & 55.64 & cm \\
Höhe KS & 1481.00 & 1611.64 & 1746.00 & 100.48 & cm \\
Länge KS & 136.00 & 218.09 & 290.00 & 41.81 & cm \\
Umfang Stämmlinge auf Höhe KS & 47.00 & 56.18 & 70.00 & 6.93 & cm \\
\bottomrule
\end{tabular}

    \caption{Feldversuch 1 - Daten der Versuchsbäume}
    \label{tab:Feldversuch_1_Versuchsbäume}
\end{table}


In [9]:
series_1.add_wind_station("06163", filename_wind=filename_wind, filename_wind_extreme=filename_wind_extreme,
                            update_existing=True)